In [1]:
# En este notebook vamos a limpiar los datos de las tablas parquet generadas en ../data_parquet/
# Comprobaremos que no haya horas con datos faltantes o duplicados, y en caso de que los haya, los corregiremos mediante interpolación temporal.
# Finalmente, guardaremos las tablas corregidas en ../data_parquet_clean/

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pathlib import Path
import shutil


In [2]:
df = pd.read_parquet("../data_parquet/embalses")
df

,AMBITO_NOMBRE,EMBALSE_NOMBRE,FECHA,AGUA_TOTAL,AGUA_ACTUAL,ELECTRICO_FLAG,porcentaje,year
0,Miño - Sil,Albarellos,2020-07-01,91.0,51.0,1,0.560440,2020
1,Miño - Sil,Albarellos,2020-04-02,91.0,52.0,1,0.571429,2020
2,Miño - Sil,Albarellos,2020-11-02,91.0,58.0,1,0.637363,2020
3,Miño - Sil,Albarellos,2020-03-03,91.0,39.0,1,0.428571,2020
4,Miño - Sil,Albarellos,2020-10-03,91.0,47.0,1,0.516484,2020
...,...,...,...,...,...,...,...,...
42946,Guadalete-Barbate,Zahara-El Gastor,2025-08-07,223.0,64.0,0,0.286996,2025
42947,Guadalete-Barbate,Zahara-El Gastor,2025-05-08,223.0,59.0,0,0.264574,2025
42948,Guadalete-Barbate,Zahara-El Gastor,2025-02-09,223.0,54.0,0,0.242152,2025
42949,Guadalete-Barbate,Zahara-El Gastor,2025-09-09,223.0,53.0,0,0.237668,2025


In [3]:
df = pd.read_parquet("../data_parquet/embalses")

print(df.info())

# ahora para cada embalse y fecha creamos un registro por hora con los mismos valores
# Asumimos que la fecha está en formato datetime y queremos expandir a horas
df['Hora'] = pd.to_datetime(df['FECHA'])  # Aseguramos que la fecha está en formato datetime

# Obtener el rango de horas para cada combinación única de AMBITO_NOMBRE, EMBALSE_NOMBRE, FECHA y year
df_expanded = []

# Agrupar por las claves
for (ambito, embalse, fecha, year), group in df.groupby(['AMBITO_NOMBRE', 'EMBALSE_NOMBRE', 'FECHA', 'year']):
    # Crear 24 registros por día para esta combinación
    fecha_dt = pd.to_datetime(fecha)
    horas = pd.date_range(start=fecha_dt.replace(hour=0), 
                         end=fecha_dt.replace(hour=23), 
                         freq='h')
    
    # Replicar los valores para cada hora
    df_day = pd.DataFrame([group.iloc[0].to_dict()] * 24)  
    df_day['Hora'] = horas  
    df_expanded.append(df_day)

# Concatenar todos los días expandidos
df_hourly = pd.concat(df_expanded, ignore_index=True)

# Ordenar el DataFrame por las claves y la hora
df_hourly = df_hourly.sort_values(['AMBITO_NOMBRE', 'EMBALSE_NOMBRE', 'year', 'Hora'])

output_dir = Path("../data_parquet_clean/embalses")
if output_dir.exists():
    shutil.rmtree(output_dir)

df_hourly.to_parquet("../data_parquet_clean/embalses", partition_cols=['year'], index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42951 entries, 0 to 42950
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   AMBITO_NOMBRE   42951 non-null  object        
 1   EMBALSE_NOMBRE  42951 non-null  object        
 2   FECHA           42951 non-null  datetime64[ns]
 3   AGUA_TOTAL      42951 non-null  float64       
 4   AGUA_ACTUAL     42951 non-null  float64       
 5   ELECTRICO_FLAG  42951 non-null  int64         
 6   porcentaje      42951 non-null  float64       
 7   year            42951 non-null  category      
dtypes: category(1), datetime64[ns](1), float64(3), int64(1), object(2)
memory usage: 2.3+ MB
None


/var/folders/z9/456rf8tn1kv_0wm71fpsqcyc0000gn/T/ipykernel_31623/1707346071.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for (ambito, embalse, fecha, year), group in df.groupby(['AMBITO_NOMBRE', 'EMBALSE_NOMBRE', 'FECHA', 'year']):


In [4]:
pd.set_option('display.max_rows', None)   
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 0)    

df_hourly[df_hourly['EMBALSE_NOMBRE'] == 'Almendra']

df_hourly.groupby(['FECHA']).count()


,AMBITO_NOMBRE,EMBALSE_NOMBRE,AGUA_TOTAL,AGUA_ACTUAL,ELECTRICO_FLAG,porcentaje,year,Hora
FECHA,,,,,,,,
2020-01-09,8640,8640,8640,8640,8640,8640,8640,8640
2020-01-12,8832,8832,8832,8832,8832,8832,8832,8832
2020-02-06,8640,8640,8640,8640,8640,8640,8640,8640
2020-03-03,8640,8640,8640,8640,8640,8640,8640,8640
2020-03-11,8832,8832,8832,8832,8832,8832,8832,8832
2020-04-02,8640,8640,8640,8640,8640,8640,8640,8640
2020-04-08,8640,8640,8640,8640,8640,8640,8640,8640
2020-05-05,8640,8640,8640,8640,8640,8640,8640,8640
2020-06-10,8832,8832,8832,8832,8832,8832,8832,8832


In [5]:
# lectura datos y función de comprobación de duplicados y faltantes


def comprobar_duplicados_faltantes(list_tabla, path="../data_parquet/"):
    faltantes_por_hora = {}
    for tabla in list_tabla:
        df = pd.read_parquet(f"{path}{tabla}")

        # Resetear el índice para tener 'Hora' como columna
        df = df.reset_index()

        # Comprobar duplicados en el campo hora con el formato datetime %Y-%m-%d %H:%M:%S   
        duplicados = df[df.duplicated(subset=['EMBALSE_NOMBRE','Hora'], keep=False)]
        if not duplicados.empty:
            print(f"Duplicados encontrados en {tabla}:")
            print(duplicados)
            print(duplicados.count())
        else:
            print(f"No se encontraron duplicados en {tabla}.")

        # Comprobar faltantes en las horas
        rango_completo = pd.date_range(start=df['Hora'].min(), end=df['Hora'].max(), freq='1h')
        faltantes = rango_completo.difference(pd.DatetimeIndex(df['Hora']))
        if not faltantes.empty:
            faltantes_por_hora[tabla] = faltantes
            print(f"Faltantes encontrados en {tabla}:")
            print(faltantes)

    return faltantes_por_hora

list_tabla = ['embalses']

comprobar_duplicados_faltantes(list_tabla,path="../data_parquet_clean/")

No se encontraron duplicados en embalses.
Faltantes encontrados en embalses:
DatetimeIndex(['2020-01-10 00:00:00', '2020-01-10 01:00:00',
               '2020-01-10 02:00:00', '2020-01-10 03:00:00',
               '2020-01-10 04:00:00', '2020-01-10 05:00:00',
               '2020-01-10 06:00:00', '2020-01-10 07:00:00',
               '2020-01-10 08:00:00', '2020-01-10 09:00:00',
               ...
               '2025-10-05 14:00:00', '2025-10-05 15:00:00',
               '2025-10-05 16:00:00', '2025-10-05 17:00:00',
               '2025-10-05 18:00:00', '2025-10-05 19:00:00',
               '2025-10-05 20:00:00', '2025-10-05 21:00:00',
               '2025-10-05 22:00:00', '2025-10-05 23:00:00'],
              dtype='datetime64[ns]', length=47568, freq=None)


{'embalses': DatetimeIndex(['2020-01-10 00:00:00', '2020-01-10 01:00:00',
                '2020-01-10 02:00:00', '2020-01-10 03:00:00',
                '2020-01-10 04:00:00', '2020-01-10 05:00:00',
                '2020-01-10 06:00:00', '2020-01-10 07:00:00',
                '2020-01-10 08:00:00', '2020-01-10 09:00:00',
                ...
                '2025-10-05 14:00:00', '2025-10-05 15:00:00',
                '2025-10-05 16:00:00', '2025-10-05 17:00:00',
                '2025-10-05 18:00:00', '2025-10-05 19:00:00',
                '2025-10-05 20:00:00', '2025-10-05 21:00:00',
                '2025-10-05 22:00:00', '2025-10-05 23:00:00'],
               dtype='datetime64[ns]', length=47568, freq=None)}

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42951 entries, 0 to 42950
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   AMBITO_NOMBRE   42951 non-null  object        
 1   EMBALSE_NOMBRE  42951 non-null  object        
 2   FECHA           42951 non-null  datetime64[ns]
 3   AGUA_TOTAL      42951 non-null  float64       
 4   AGUA_ACTUAL     42951 non-null  float64       
 5   ELECTRICO_FLAG  42951 non-null  int64         
 6   porcentaje      42951 non-null  float64       
 7   year            42951 non-null  category      
 8   Hora            42951 non-null  datetime64[ns]
dtypes: category(1), datetime64[ns](2), float64(3), int64(1), object(2)
memory usage: 2.7+ MB
